In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/SA

/content/drive/MyDrive/SA


In [ ]:
!pip install gensim==4.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 2.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim
import random
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchtext
from torchtext.data import get_tokenizer
import re
import nltk
from torch.utils.data import TensorDataset, DataLoader
from nltk.corpus import stopwords 
from collections import Counter
from LSTM import DLSTM, CustomLSTM
from BILSTM import DBLSTM
from torch.utils.tensorboard import SummaryWriter
from w2v import w2v_prep
from Attent import Att
writer = SummaryWriter()
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = pd.read_csv('data_imdb.csv')
df = df.head(5000)
review = df['review'].to_list()
sentiment = df['sentiment'].to_list()
y = [0] * len(sentiment)
for i in range(len(sentiment)):
    if sentiment[i] == 'positive':
        y[i] = 1
df1 = pd.DataFrame(data=np.array(y).T, columns=['sentiment'])

In [ ]:
ED = 50
seq_len = 100
w2v_model, review_text, vocab_len, X_train, X_val, X_test, Y_train, Y_val, Y_test = w2v_prep(df, df1, ED)

In [ ]:
weights = torch.FloatTensor(w2v_model.wv.vectors)
vocab = w2v_model.wv.key_to_index
print(len(vocab))
vocab[' '] = len(vocab) 

12188


In [ ]:
weights = torch.cat((weights,torch.tensor(([ED*[0]]),dtype=float)),dim=0)
print(weights.shape)

torch.Size([12189, 50])


In [ ]:
def tockenize(vocab,seq_len,x_vec):
    word_list = []  
    
    final_token = np.array([],dtype = int)  
    count=0
    
    for sent in x_vec:
        sent_len = len(sent)
        tokens = [vocab[word] for word in sent if word in vocab.keys()]
        
        # CALCULATING TOKENS
        if len(tokens) >= seq_len:
            temp_vec = np.array(tokens[0:seq_len])
        else:
            temp_vec = np.array(tokens + (seq_len - len(tokens)) * [vocab[' ']])
            
        # STACKING   
        if count==0:
            final_token = np.hstack((final_token,temp_vec))
            count+=1
        else:
            final_token = np.vstack((final_token,temp_vec))
        
    return final_token

In [ ]:
x_train = tockenize(vocab,seq_len,X_train)
np.save('x_train',x_train)
x_val = tockenize(vocab,seq_len,X_val)
np.save('x_val',x_val)
x_test = tockenize(vocab,seq_len,X_test)
np.save('x_test',x_test)

In [ ]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
x_val = np.load('x_val.npy')

In [ ]:
x_train.shape

(3500, 100)

In [ ]:
batch_size = 100
train_data = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(np.array(Y_train)))
test_data = TensorDataset(torch.from_numpy(x_test), torch.from_numpy(np.array(Y_test)))
valid_data = TensorDataset(torch.from_numpy(x_val), torch.from_numpy(np.array(Y_val)))
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


In [ ]:
class SentimentRNN(nn.Module):
    def __init__(self, weights, output_dim, vocab_size, hidden_dim, embedding_dim, batch_size, seq_len, n_layers,verbose):
        super(SentimentRNN, self).__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.embedding_len = embedding_dim

        # embedding and LSTM layers
        #self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # With pretrained embeddings
        #weights = torch.FloatTensor(w2v_model.wv.vectors)
        self.embedding = nn.Embedding.from_pretrained(weights)

        # lstm
        self.lstm = DBLSTM(self.embedding_len, self.hidden_dim, self.batch_size, seq_len, n_layers, verbose)
        self.att = Att( 2*self.hidden_dim, self.batch_size, seq_len)

        # linear and sigmoid layer
        self.fc = nn.Linear(2*self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        bs = x.size(0)
        
        # Embedding Layer
        embeds = self.embedding(x)
        embeds = embeds.float()
        # LSTM Layer
        y_t,y_net = self.lstm(embeds)
        
        #Attention
        y_t = self.att(y_net)

        #x = (x[:, -1, :])

        # Fully connected Layer
        out = self.fc(y_t)

        # Sigmoid Function
        sig_out = self.sig(out)

        # reshape to be batch_size first
        sig_out = sig_out.view(bs, -1)
        

        sig_out = sig_out[:, -1]  # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out
def acc(pred, label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [ ]:
n_layers = 2
output_dim = 1
hidden_dim = 256
lr = 0.001
vocab_len+=1
verbose = True
criterion = nn.BCELoss()
model = SentimentRNN(weights, output_dim, vocab_len, hidden_dim, ED, batch_size, seq_len, n_layers,verbose)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
print(model)

SentimentRNN(
  (embedding): Embedding(12189, 50)
  (lstm): DBLSTM(
    (lstm1): BILSTM()
    (dlstm): ModuleList(
      (0): BVLSTM()
    )
  )
  (att): Att(
    (softmax): Softmax(dim=0)
  )
  (fc): Linear(in_features=512, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
epochs = 10
valid_loss_min = np.Inf
epoch_tr_loss, epoch_vl_loss = [], []
epoch_tr_acc, epoch_vl_acc = [], []
for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    count =0
    for inputs, labels in train_loader:
        #print("Epoch",epoch,"batch",count+1)
        inputs, labels = inputs.to(device), labels.to(device)
        #print(inputs.shape)
        #print(labels.shape)
        optimizer.zero_grad()
        output = model(inputs)
        print(output.shape)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        accuracy = acc(output, labels)
        train_acc += accuracy
        writer.add_scalar("Train Loss", loss, epoch)
        count+=1
    writer.flush()
    val_losses = []
    val_acc = 0
    model.eval()
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs)
        val_loss = criterion(output.squeeze(), labels.float())
        val_losses.append(val_loss.item())
        accuracy = acc(output, labels)
        val_acc += accuracy
        writer.add_scalar("Train Loss", val_loss, epoch)
    writer.flush()
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc / len(train_loader.dataset)
    epoch_val_acc = val_acc / len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch + 1}')
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc * 100} val_accuracy : {epoch_val_acc * 100}')
    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), 'my_model_best_w2v.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25 * '==')
writer.close()

torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
Epoch 1
train_loss : 0.7471551503453936 val_loss : 0.7024542927742005
train_accuracy : 51.34285714285715 val_accuracy : 51.2
Validation loss decreased (inf --> 0.702454).  Saving model ...
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
t

KeyboardInterrupt: ignored

In [ ]:
def test_results():
    test_losses = []
    test_acc = 0.0
    model.eval()
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        output = model(inputs)
        test_loss = criterion(output.squeeze(), labels.float())

        test_losses.append(test_loss.item())

        accuracy = acc(output, labels)
        test_acc += accuracy
    print("Test Accuracy : ", test_acc / len(test_loader.dataset))

test_results()

***Attention Module***

```
import torch
import torch.nn as nn
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Att(nn.Module):
  def __init__(self, hidden_size, batch_size, seq_len):
        super().__init__()
        self.input_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len


        self.w = nn.Parameter(torch.Tensor(self.input_size))
        self.b = nn.Parameter(torch.Tensor(self.seq_len))
        self.softmax = nn.Softmax(dim =0)

        self.init_weight()

  def init_weight(self):
      stdv = 1.0 / math.sqrt(self.seq_len)
      for weights in self.parameters():
          weights.data.uniform_(-stdv, stdv)

  def forward(self,x):
    et = torch.tanh(x @ self.w + self.b)
    #print(et.shape)
    at = self.softmax(et).unsqueeze(1)
    #print(at.shape)
    output = at @ x
    output = output.squeeze(1)
    #print(output.shape)
    return output
```



***BI LSTM***



```
import torch
import torch.nn as nn
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class BILSTM(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, seq_len):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len

        #Forward
        # i_t
        self.w_i = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_i = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(self.hidden_size))
        # f_t
        self.w_f = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_f = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(self.hidden_size))
        # c_t
        self.w_c = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_c = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(self.hidden_size))
        # o_t
        self.w_o = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_o = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(self.hidden_size))
        

        #Forward
        # i_t
        self.w_i_b = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_i_b = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_i_b = nn.Parameter(torch.Tensor(self.hidden_size))
        # f_t
        self.w_f_b = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_f_b = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_f_b = nn.Parameter(torch.Tensor(self.hidden_size))
        # c_t
        self.w_c_b = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_c_b = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_c_b = nn.Parameter(torch.Tensor(self.hidden_size))
        # o_t
        self.w_o_b = nn.Parameter(torch.Tensor(self.input_size, self.hidden_size))
        self.u_o_b = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_o_b = nn.Parameter(torch.Tensor(self.hidden_size))

        self.init_weight()

    def init_weight(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weights in self.parameters():
            weights.data.uniform_(-stdv, stdv)

    def forward(self, x, init_states=None):
        batch_size, seq_len = self.batch_size, self.seq_len
        hidden_seq = []
        hidden_seq_b = []
        y_net = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size).to(x.device)
            c_t = torch.zeros(batch_size,self.hidden_size).to(x.device)
            h_t_b = torch.zeros(batch_size, self.hidden_size).to(x.device)
            c_t_b = torch.zeros(batch_size,self.hidden_size).to(x.device)
        
        else:
            h_t, c_t, h_t_b, c_t_b = init_states

        for t in range(seq_len):
            x_t = x[:, t, :]
            i_t = torch.sigmoid(x_t @ self.w_i + h_t @ self.u_i + self.b_i)
            f_t = torch.sigmoid(x_t @ self.w_f + h_t @ self.u_f + self.b_f)
            g_t = torch.sigmoid(x_t @ self.w_c + h_t @ self.u_c + self.b_c)
            o_t = torch.sigmoid(x_t @ self.w_o + h_t @ self.u_o + self.b_o)
            c_t = f_t + c_t + i_t * g_t
            h_t = o_t + torch.tanh(c_t)

            x_t_b = x[:, seq_len-t-1, :]
            i_t_b = torch.sigmoid(x_t_b @ self.w_i_b + h_t_b @ self.u_i_b + self.b_i_b)
            f_t_b = torch.sigmoid(x_t_b @ self.w_f_b + h_t_b @ self.u_f_b + self.b_f_b)
            g_t_b = torch.sigmoid(x_t_b @ self.w_c_b + h_t_b @ self.u_c_b + self.b_c_b)
            o_t_b = torch.sigmoid(x_t_b @ self.w_o_b + h_t_b @ self.u_o_b + self.b_o_b)
            c_t_b = f_t_b + c_t_b + i_t_b * g_t_b
            h_t_b = o_t_b + torch.tanh(c_t_b)

            hidden_seq.append(h_t.unsqueeze(0))
            hidden_seq_b.append(h_t_b.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq_b = torch.cat(hidden_seq_b, dim=0)
        
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        hidden_seq_b = hidden_seq_b.transpose(0, 1).contiguous()
        hidden_seq_b = torch.flip(hidden_seq_b,dims=(1,))
        
        y_net = torch.cat((hidden_seq,hidden_seq_b),dim=2)
        y_t = torch.cat((h_t,h_t_b),dim=1)

        return y_t, y_net
    
    
class BVLSTM(nn.Module):
    def __init__(self, input_size_x, input_size_h, hidden_size, batch_size, seq_len):
        super().__init__()
        self.input_size_x = input_size_x
        self.input_size_h = input_size_h
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len

        # i_t
        self.w_i_x = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_i_h = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_i = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_i = nn.Parameter(torch.Tensor(self.hidden_size))
        # f_t
        self.w_f_x = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_f_h = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_f = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_f = nn.Parameter(torch.Tensor(self.hidden_size))
        # c_t
        self.w_c_x = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_c_h = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_c = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_c = nn.Parameter(torch.Tensor(self.hidden_size))
        # o_t
        self.w_o_x = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_o_h = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_o = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_o = nn.Parameter(torch.Tensor(self.hidden_size))

        #Backward
        # i_t
        self.w_i_x_b = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_i_h_b = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_i_b  = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_i_b  = nn.Parameter(torch.Tensor(self.hidden_size))
        # f_t
        self.w_f_x_b  = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_f_h_b  = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_f_b  = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_f_b  = nn.Parameter(torch.Tensor(self.hidden_size))
        # c_t
        self.w_c_x_b  = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_c_h_b = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_c_b  = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_c_b  = nn.Parameter(torch.Tensor(self.hidden_size))
        # o_t
        self.w_o_x_b  = nn.Parameter(torch.Tensor(self.input_size_x, self.hidden_size))
        self.w_o_h_b  = nn.Parameter(torch.Tensor(self.input_size_h, self.hidden_size))
        self.u_o_b  = nn.Parameter(torch.Tensor(self.hidden_size, self.hidden_size))
        self.b_o_b  = nn.Parameter(torch.Tensor(self.hidden_size))

        self.init_weight()

    def init_weight(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weights in self.parameters():
            weights.data.uniform_(-stdv, stdv)

    def forward(self, x, h_prev, init_states=None):
        batch_size, seq_len = self.batch_size, self.seq_len
        hidden_seq = []
        hidden_seq_b = []
        y_net = []
        h_prev_b = h_prev[:,:,self.hidden_size:2*self.hidden_size]
        h_prev = h_prev[:,:,0:self.hidden_size]

        if init_states is None:
            h_t, c_t = torch.zeros(batch_size, self.hidden_size).to(x.device), torch.zeros(batch_size,
                                                                                           self.hidden_size).to(
                x.device)
            h_t_b = torch.zeros(batch_size, self.hidden_size).to(x.device)
            c_t_b = torch.zeros(batch_size,self.hidden_size).to(x.device)
        else:
            h_t, c_t, h_t_b, c_t_b = init_states

        for t in range(seq_len):
            x_t = x[:, t, :]
            h_prev_t = h_prev[:, t, :]
            i_t = torch.sigmoid(x_t @ self.w_i_x + h_prev_t @ self.w_i_h  + h_t @ self.u_i + self.b_i)
            f_t = torch.sigmoid(x_t @ self.w_f_x + h_prev_t @ self.w_f_h  + h_t @ self.u_i + self.b_i)
            g_t = torch.sigmoid(x_t @ self.w_c_x + h_prev_t @ self.w_c_h  + h_t @ self.u_i + self.b_i)
            o_t = torch.sigmoid(x_t @ self.w_o_x + h_prev_t @ self.w_o_h  + h_t @ self.u_i + self.b_i)
            c_t = f_t + c_t + i_t * g_t
            h_t = o_t + torch.tanh(c_t)


            x_t_b = x[:, seq_len-t-1, :]
            h_prev_t_b = h_prev_b[:, seq_len-t-1, :]
            i_t_b = torch.sigmoid(x_t_b @ self.w_i_x_b + h_prev_t_b @ self.w_i_h_b  + h_t_b @ self.u_i_b + self.b_i_b)
            f_t_b = torch.sigmoid(x_t_b @ self.w_f_x_b + h_prev_t_b @ self.w_f_h_b  + h_t_b @ self.u_i_b + self.b_i_b)
            g_t_b = torch.sigmoid(x_t_b @ self.w_c_x_b + h_prev_t_b @ self.w_c_h_b  + h_t_b @ self.u_i_b + self.b_i_b)
            o_t_b = torch.sigmoid(x_t_b @ self.w_o_x_b + h_prev_t_b @ self.w_o_h_b  + h_t_b @ self.u_i_b + self.b_i_b)
            c_t_b = f_t_b + c_t_b + i_t_b * g_t_b
            h_t_b = o_t_b + torch.tanh(c_t_b)


            hidden_seq.append(h_t.unsqueeze(0))
            hidden_seq_b.append(h_t_b.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq_b = torch.cat(hidden_seq_b, dim=0)
        
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        hidden_seq_b = hidden_seq_b.transpose(0, 1).contiguous()
        hidden_seq_b = torch.flip(hidden_seq_b,dims=(1,))
        
        y_net = torch.cat((hidden_seq,hidden_seq_b),dim=2)
        y_t = torch.cat((h_t,h_t_b),dim=1)

        return y_t, y_net


class DBLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, seq_len, n_layers, verbose  = True):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        self.n_layers = n_layers
        self.verbose = verbose
        self.lstm1 = BILSTM(input_size, hidden_size, batch_size, seq_len)
        
        if self.verbose:
            self.dlstm = nn.ModuleList(
            [BVLSTM(input_size, hidden_size, hidden_size, batch_size, seq_len) for _ in range(self.n_layers-1)]
            )
        else:
            self.dlstm = nn.ModuleList(
            [BILSTM(hidden_size, hidden_size, batch_size, seq_len) for _ in range(self.n_layers-1)]
            )
            

    def forward(self, x):
        y_t, y_net = self.lstm1(x)
        if self.verbose: 
            for lstm in self.dlstm:
                y_t, y_net = lstm(x, y_net)
        else:
            for lstm in self.dlstm:
                y_t, y_net = lstm(y_net)
        return y_t, y_net

```



***w2v.py***



```
import gensim
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec


def w2v_prep(df, df1, Embedding_dimensions=100):
    review_text = df.review.apply(gensim.utils.simple_preprocess)
    w2v_model = Word2Vec(review_text, vector_size=Embedding_dimensions, workers=8, min_count=5)
    vocab_len = len(w2v_model.wv.key_to_index)
    #print(w2v_model.wv.key_to_index)
    
    X_train, X_test, Y_train, Y_test = train_test_split(review_text,
                                                        df1['sentiment'],
                                                        test_size=0.2,
                                                        random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train,
                                                      Y_train,
                                                      test_size=0.125,
                                                      random_state=0)
    return w2v_model, review_text, vocab_len, X_train, X_val, X_test, Y_train, Y_val, Y_test

```

